# LOTUS - dataset overview

[LOTUS initiative](https://lotus.nprod.net/) is a database of natural products connected to organisms. Befor expanding the database, we like to have an overview about the last LOTUS datasest.

[latest LOTUS dataset (v10)](https://zenodo.org/records/7534071)

More info in the [Paper (DOI:70780)](https://elifesciences.org/articles/70780)

## load dataset

In [2]:
# Example of loading LOTUS datasets with polars (python module)
import polars as pl
import numpy as np
from defl import *  # module for handling LOTUS and MINES

df_lotus = pl.read_parquet("../data/LOTUS/230106_frozen_metadata_cleaned.parquet")
print(f"all columns of LOTUS (total: {df_lotus.shape[1]}): \n{df_lotus.columns}")

all columns of LOTUS (total: 39): 
['structure_wikidata', 'structure_inchikey', 'structure_inchi', 'structure_smiles', 'structure_molecular_formula', 'structure_exact_mass', 'structure_xlogp', 'structure_smiles_2D', 'structure_cid', 'structure_nameIupac', 'structure_nameTraditional', 'structure_stereocenters_total', 'structure_stereocenters_unspecified', 'structure_taxonomy_npclassifier_01pathway', 'structure_taxonomy_npclassifier_02superclass', 'structure_taxonomy_npclassifier_03class', 'structure_taxonomy_classyfire_chemontid', 'structure_taxonomy_classyfire_01kingdom', 'structure_taxonomy_classyfire_02superclass', 'structure_taxonomy_classyfire_03class', 'structure_taxonomy_classyfire_04directparent', 'organism_wikidata', 'organism_name', 'organism_taxonomy_gbifid', 'organism_taxonomy_ncbiid', 'organism_taxonomy_ottid', 'organism_taxonomy_01domain', 'organism_taxonomy_02kingdom', 'organism_taxonomy_03phylum', 'organism_taxonomy_04class', 'organism_taxonomy_05order', 'organism_taxono

In [5]:
# example of "limonene" entry

df = df_lotus.filter((pl.col("structure_nameTraditional") == "Limonene") & (pl.col("organism_taxonomy_09species") == "Cannabis sativa"))[0, :]

#df.write_csv("Limonene.csv", separator=",")

The 39 columns are separated in **natural products (NP)**, **chemical structure**,  **organism** and the **source**.  

In [11]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

for columnname in df_lotus.columns:
    # count the appearance of each "word" in the given column
    df_lotus_plot = df_lotus.select(pl.col(columnname).value_counts(sort=True, name="n")).unnest(columnname)

    # print the 5 most common names
    print(f"{columnname} (shape: {df_lotus_plot.shape[0]:_}) : {df_lotus_plot[0:5, :]}")

structure_wikidata (shape: 220_783) : shape: (5, 2)
┌────────────────────────────────────────┬──────┐
│ structure_wikidata                     ┆ n    │
│ ---                                    ┆ ---  │
│ str                                    ┆ u32  │
╞════════════════════════════════════════╪══════╡
│ http://www.wikidata.org/entity/Q209727 ┆ 4156 │
│ http://www.wikidata.org/entity/Q121802 ┆ 3142 │
│ http://www.wikidata.org/entity/Q425004 ┆ 3006 │
│ http://www.wikidata.org/entity/Q409478 ┆ 2390 │
│ http://www.wikidata.org/entity/Q278809 ┆ 2294 │
└────────────────────────────────────────┴──────┘
structure_inchikey (shape: 220_823) : shape: (5, 2)
┌─────────────────────────────┬──────┐
│ structure_inchikey          ┆ n    │
│ ---                         ┆ ---  │
│ str                         ┆ u32  │
╞═════════════════════════════╪══════╡
│ IPCSVZSSVZVIGE-UHFFFAOYSA-N ┆ 4156 │
│ KZJWDPNRJALLNS-VJSFXXLFSA-N ┆ 3142 │
│ HCXVJBMSMIARIN-PHZDYDNGSA-N ┆ 3006 │
│ REFJWTPEDVJJIY-UHFFFAOYSA-N ┆ 23

## Inputfile (Wikidata and SMILES)

For the pickaxe (MINEs) it is necessary to provide a SMILE and a ID.  
In this case would be used the coloumns **3D SMILES (structure_smiles)** and **structure_wikidata**.  

In [3]:
# write in bold the specific title
print("\033[1m\nuniqueness over categories:\033[0m")

# unique counts only of the "chemical structure" columns
unique_counts = df_lotus.select(
    df_lotus.columns[0:20]
    ).n_unique()
print("all structure columns:", str(unique_counts))

# unique counts only of the "organism" columns []
unique_counts = df_lotus.select(
    df_lotus.columns[21:35]
    ).n_unique()
print("all organism columns:", str(unique_counts))

# unique counts for the full dataset
unique_counts = df_lotus.select(
    df_lotus.columns[:]
    ).n_unique()
print("all columns:", str(unique_counts))

# unique counts for the pickaxe/MINEs input
print("\033[1m\nImportant Info for pickaxe (MINEs):\033[0m")
print(f'rows for LOTUS dataset: {df_lotus.shape[0]}')
print(f'\nunique "structure_smiles": {df_lotus.select(["structure_smiles"]).n_unique()}')
print(f'unique "structure_wikidata": {df_lotus.select(["structure_wikidata"]).n_unique()}')
print(f'unique "structure_smiles" and "structure_wikidata": {df_lotus.select(["structure_smiles", "structure_wikidata"]).n_unique()}')
print(f'\nunique "structure_smiles": {df_lotus.select(["structure_smiles"]).n_unique()}')
print(f'unique "structure_inchi": {df_lotus.select(["structure_inchi"]).n_unique()}')
print(f'unique "structure_smiles" and "structure_inchi": {df_lotus.select(["structure_smiles", "structure_inchi"]).n_unique()}')


uniqueness over categories:
all structure columns: 257226
all organism columns: 36803
all columns: 792364

Important Info for pickaxe (MINEs):
rows for LOTUS dataset: 792364

unique "structure_smiles": 220820
unique "structure_wikidata": 220783
unique "structure_smiles" and "structure_wikidata": 220834

unique "structure_smiles": 220820
unique "structure_inchi": 220823
unique "structure_smiles" and "structure_inchi": 220823


The uniqueness will be reduced from the full dataset (792'364 rows) to the pickaxe inputfile "structure_smiles" and "structure_wikidata" (220'834 rows), which is **27.9%** of the full dataset.  

This are good news, so the dataset will be reduced drasticaly.  

Which is a little unusual that we have more SMILES than wikidata links. 
This could be, because not all SMILES have a wikidata link yet, which in turn cannot be because LOTUS is ‘complete’ and has no empty fields.  

If we look at the inchi's, it is also a little bit surprising, that there should be 3 duplicates. 

In [4]:
# make a dataframe with only the two columns: "structure_smiles" and "structure_wikidata"
df_lotus_for_pickaxe_with_wikidata = df_lotus.select(["structure_smiles", "structure_wikidata"]).unique()

# search for the duplicates and print them
df_lotus_for_pickaxe_with_wikidata_duplicates = df_lotus_for_pickaxe_with_wikidata.filter(df_lotus_for_pickaxe_with_wikidata.select(['structure_smiles']).is_duplicated()).sort('structure_smiles')
print(f'{df_lotus_for_pickaxe_with_wikidata_duplicates[0:2, :]}')


# make a dataframe with only the two columns: "structure_smiles" and "structure_inchi"
df_lotus_for_pickaxe_with_inchi = df_lotus.select(["structure_smiles", "structure_inchi"]).unique()

# search for the duplicates and print them
df_lotus_for_pickaxe_with_inchi_duplicates = df_lotus_for_pickaxe_with_inchi.filter(df_lotus_for_pickaxe_with_inchi.select(['structure_smiles']).is_duplicated()).sort('structure_smiles')
print(f'{df_lotus_for_pickaxe_with_inchi_duplicates[0:2, :]}')


shape: (2, 2)
┌───────────────────┬──────────────────────────────────────────┐
│ structure_smiles  ┆ structure_wikidata                       │
│ ---               ┆ ---                                      │
│ str               ┆ str                                      │
╞═══════════════════╪══════════════════════════════════════════╡
│ C1CCC2(CCCCO2)OC1 ┆ http://www.wikidata.org/entity/Q55620521 │
│ C1CCC2(CCCCO2)OC1 ┆ http://www.wikidata.org/entity/Q804105   │
└───────────────────┴──────────────────────────────────────────┘
shape: (2, 2)
┌─────────────────────────────────────────────────┬────────────────────────────────────────────────┐
│ structure_smiles                                ┆ structure_inchi                                │
│ ---                                             ┆ ---                                            │
│ str                                             ┆ str                                            │
╞═══════════════════════════════════════════════

The real reason is, that some SMILES have two wikidata entries, which is confusing. If we check the links, they are both linked to the "same" chemical component.
This makes also sense, because the amount of this duplicated entries (28) correspond to the amount of entries of only SMILES (220820) plus the additinal entries (14).

The same problem we can see with the INCHI's. In this case, we have multiple inchi's for the same SMILE's. This is possible, because the INCHI's describes the molecule more exactly then the SMILES.

### Why do we have less wikidata links then SMILES? This is a lose of information, no?  
We can see below, that some wikidatalinks are pointing to multiple SMILE's. This can be the case, because the wikilinks sometimes are represanting chemical groupes.

In [5]:
# make a dataframe with only the two columns: "structure_smiles" and "structure_wikidata"
df_lotus_for_pickaxe_with_wikidata = df_lotus.select(["structure_smiles", "structure_wikidata"]).unique()

# search for the duplicates and print them
df_lotus_for_pickaxe_with_wikidata_duplicates = df_lotus_for_pickaxe_with_wikidata.filter(df_lotus_for_pickaxe_with_wikidata.select(['structure_wikidata']).is_duplicated()).sort('structure_wikidata')
print(f'{df_lotus_for_pickaxe_with_wikidata_duplicates[0:2, :]}')

shape: (2, 2)
┌──────────────────────────────────────────────────────┬───────────────────────────────────────────┐
│ structure_smiles                                     ┆ structure_wikidata                        │
│ ---                                                  ┆ ---                                       │
│ str                                                  ┆ str                                       │
╞══════════════════════════════════════════════════════╪═══════════════════════════════════════════╡
│ C/C=C1/CN2CC[C@@]34c5ccccc5N5[C@H](C(=O)OC)[C@H]1C[C ┆ http://www.wikidata.org/entity/Q105144092 │
│ @H]2[C@]53Oc1c4cc2c(c1O)C(=O)O[C@]13[C@@H]4C[C@H]5/C ┆                                           │
│ (=C\C)CN4CC[C@@]21c1ccccc1N3[C@@H]5C(=O)OC           ┆                                           │
│ C/C=C1/CN2CC[C@]34c5ccccc5N5[C@H](C(=O)OC)[C@H]1C[C@ ┆ http://www.wikidata.org/entity/Q105144092 │
│ H]2[C@@]53Oc1c4cc2c(c1O)C(=O)O[C@@]13[C@@H]4C[C@H]5/ ┆                     

## Inputfile (InchyKey and SMILES)

In [5]:
df_lotus.columns

['structure_wikidata',
 'structure_inchikey',
 'structure_inchi',
 'structure_smiles',
 'structure_molecular_formula',
 'structure_exact_mass',
 'structure_xlogp',
 'structure_smiles_2D',
 'structure_cid',
 'structure_nameIupac',
 'structure_nameTraditional',
 'structure_stereocenters_total',
 'structure_stereocenters_unspecified',
 'structure_taxonomy_npclassifier_01pathway',
 'structure_taxonomy_npclassifier_02superclass',
 'structure_taxonomy_npclassifier_03class',
 'structure_taxonomy_classyfire_chemontid',
 'structure_taxonomy_classyfire_01kingdom',
 'structure_taxonomy_classyfire_02superclass',
 'structure_taxonomy_classyfire_03class',
 'structure_taxonomy_classyfire_04directparent',
 'organism_wikidata',
 'organism_name',
 'organism_taxonomy_gbifid',
 'organism_taxonomy_ncbiid',
 'organism_taxonomy_ottid',
 'organism_taxonomy_01domain',
 'organism_taxonomy_02kingdom',
 'organism_taxonomy_03phylum',
 'organism_taxonomy_04class',
 'organism_taxonomy_05order',
 'organism_taxonomy_0

In [7]:
# write in bold the specific title
print("\033[1m\nuniqueness over categories:\033[0m")

# unique counts only of the "chemical structure" columns
unique_counts = df_lotus.select(
    df_lotus.columns[0:20]
    ).n_unique()
print("all structure columns:", str(unique_counts))

# unique counts only of the "organism" columns []
unique_counts = df_lotus.select(
    df_lotus.columns[21:35]
    ).n_unique()
print("all organism columns:", str(unique_counts))

# unique counts for the full dataset
unique_counts = df_lotus.select(
    df_lotus.columns[:]
    ).n_unique()
print("all columns:", str(unique_counts))

# unique counts for the pickaxe/MINEs input
print("\033[1m\nImportant Info for pickaxe (MINEs):\033[0m")
print(f'unique "structure_smiles": {df_lotus.select(["structure_smiles"]).n_unique()}')
print(f'unique "structure_inchikey": {df_lotus.select(["structure_inchikey"]).n_unique()}')
print(f'unique "structure_smiles" and "structure_inchikey": {df_lotus.select(["structure_smiles", "structure_inchikey"]).n_unique()}')


uniqueness over categories:
all structure columns: 257226
all organism columns: 36803
all columns: 792364

Important Info for pickaxe (MINEs):
unique "structure_smiles": 220820
unique "structure_inchikey": 220823
unique "structure_smiles" and "structure_inchikey": 220823


In [9]:
# make a dataframe with only the two columns: "structure_smiles" and "structure_inchi"
df_lotus_for_pickaxe_with_inchikey = df_lotus.select(["structure_smiles", "structure_inchikey"]).unique()

# search for the duplicates and print them
df_lotus_for_pickaxe_with_inchikey_duplicates = df_lotus_for_pickaxe_with_inchikey.filter(df_lotus_for_pickaxe_with_inchikey.select(['structure_smiles']).is_duplicated()).sort('structure_smiles')
print(f'{df_lotus_for_pickaxe_with_inchikey_duplicates}')

shape: (6, 2)
┌────────────────────────────────────────────────────────────────────┬─────────────────────────────┐
│ structure_smiles                                                   ┆ structure_inchikey          │
│ ---                                                                ┆ ---                         │
│ str                                                                ┆ str                         │
╞════════════════════════════════════════════════════════════════════╪═════════════════════════════╡
│ CC1=C\C=C\C(C)=C\C[C@H](C)NC(=O)/C(CC(C)C)=C/C(C)=C/C=C/C=C/[C@@]( ┆ SCXSAOWAWJDUND-ROXKSDJVSA-N │
│ C)(O)[C@@H](O[C@@H]2OC[C@@H](O[C@H]3C[C@@](C)(O)[C@H](N(C)C)[C@@H] ┆                             │
│ (C)O3)[C@H](O)[C@H]2N)/C=C\C=C\1                                   ┆                             │
│ CC1=C\C=C\C(C)=C\C[C@H](C)NC(=O)/C(CC(C)C)=C/C(C)=C/C=C/C=C/[C@@]( ┆ SCXSAOWAWJDUND-ZBTULVDDSA-N │
│ C)(O)[C@@H](O[C@@H]2OC[C@@H](O[C@H]3C[C@@](C)(O)[C@H](N(C)C)[C@@H] ┆       

## filter


In [3]:
df_lotus.filter(pl.col("structure_taxonomy_classyfire_04directparent") == "Flavonoid-3-O-glycosides")

structure_wikidata,structure_inchikey,structure_inchi,structure_smiles,structure_molecular_formula,structure_exact_mass,structure_xlogp,structure_smiles_2D,structure_cid,structure_nameIupac,structure_nameTraditional,structure_stereocenters_total,structure_stereocenters_unspecified,structure_taxonomy_npclassifier_01pathway,structure_taxonomy_npclassifier_02superclass,structure_taxonomy_npclassifier_03class,structure_taxonomy_classyfire_chemontid,structure_taxonomy_classyfire_01kingdom,structure_taxonomy_classyfire_02superclass,structure_taxonomy_classyfire_03class,structure_taxonomy_classyfire_04directparent,organism_wikidata,organism_name,organism_taxonomy_gbifid,organism_taxonomy_ncbiid,organism_taxonomy_ottid,organism_taxonomy_01domain,organism_taxonomy_02kingdom,organism_taxonomy_03phylum,organism_taxonomy_04class,organism_taxonomy_05order,organism_taxonomy_06family,organism_taxonomy_07tribe,organism_taxonomy_08genus,organism_taxonomy_09species,organism_taxonomy_10varietas,reference_wikidata,reference_doi,manual_validation
str,str,str,str,str,f64,f64,str,i64,str,str,i64,i64,str,str,str,i64,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str
"""http://www.wikidata.org/entity/Q407857""","""IKGXIBQEEMLURG-NVPNHPEKSA-N""","""InChI=1S/C27H30O16/c1-8-17(32)20(35)22(37)26(40-8)39-7-15-18(33)21(36)23(38)27(42-15)43-25-19(34)16-13(31)5-10(28)6-14(16)41-24(25)9-2-3-11(29)12(30)4-9/h2-6,8,15,17-18,20-23,26-33,35-38H,7H2,1H3/t8-,15+,17-,18+,20+,21-,22+,23+,26+,27-/m0/s1""","""C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](Oc3c(-c4ccc(O)c(O)c4)oc4cc(O)cc(O)c4c3=O)[C@H](O)[C@@H](O)[C@@H]2O)[C@H](O)[C@H](O)[C@H]1O""","""C27H30O16""",610.153385,-1.6871,"""CC1OC(OCC2OC(Oc3c(-c4ccc(O)c(O)c4)oc4cc(O)cc(O)c4c3=O)C(O)C(O)C2O)C(O)C(O)C1O""",5280805,"""2-(3,4-dihydroxyphenyl)-5,7-dihydroxy-3-[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-[[(2R,3R,4R,5R,6S)-3,4,5-trihydroxy-6-methyloxan-2-yl]oxymethyl]oxan-2-yl]oxychromen-4-one""","""Rutin""",10,0,"""Shikimates and Phenylpropanoids""","""Flavonoids""","""Flavonols""",3531,"""Organic compounds""","""Phenylpropanoids and polyketides""","""Flavonoids""","""Flavonoid-3-O-glycosides""","""http://www.wikidata.org/entity/Q15232815""","""Nerium indicum""",null,null,65881,"""Eukaryota""","""Archaeplastida""","""Streptophyta""","""Magnoliopsida""","""Gentianales""","""Apocynaceae""","""Nerieae""","""Nerium""","""Nerium oleander""",null,"""http://www.wikidata.org/entity/Q104384464""","""10.1016/0031-9422(95)00837-3""",null
"""http://www.wikidata.org/entity/Q407857""","""IKGXIBQEEMLURG-NVPNHPEKSA-N""","""InChI=1S/C27H30O16/c1-8-17(32)20(35)22(37)26(40-8)39-7-15-18(33)21(36)23(38)27(42-15)43-25-19(34)16-13(31)5-10(28)6-14(16)41-24(25)9-2-3-11(29)12(30)4-9/h2-6,8,15,17-18,20-23,26-33,35-38H,7H2,1H3/t8-,15+,17-,18+,20+,21-,22+,23+,26+,27-/m0/s1""","""C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](Oc3c(-c4ccc(O)c(O)c4)oc4cc(O)cc(O)c4c3=O)[C@H](O)[C@@H](O)[C@@H]2O)[C@H](O)[C@H](O)[C@H]1O""","""C27H30O16""",610.153385,-1.6871,"""CC1OC(OCC2OC(Oc3c(-c4ccc(O)c(O)c4)oc4cc(O)cc(O)c4c3=O)C(O)C(O)C2O)C(O)C(O)C1O""",5280805,"""2-(3,4-dihydroxyphenyl)-5,7-dihydroxy-3-[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-[[(2R,3R,4R,5R,6S)-3,4,5-trihydroxy-6-methyloxan-2-yl]oxymethyl]oxan-2-yl]oxychromen-4-one""","""Rutin""",10,0,"""Shikimates and Phenylpropanoids""","""Flavonoids""","""Flavonols""",3531,"""Organic compounds""","""Phenylpropanoids and polyketides""","""Flavonoids""","""Flavonoid-3-O-glycosides""","""http://www.wikidata.org/entity/Q155833""","""Nerium oleander""",null,63479,65881,"""Eukaryota""","""Archaeplastida""","""Streptophyta""","""Magnoliopsida""","""Gentianales""","""Apocynaceae""","""Nerieae""","""Nerium""","""Nerium oleander""",null,"""http://www.wikidata.org/entity/Q104384464""","""10.1016/0031-9422(95)00837-3""",null
"""http://www.wikidata.org/entity/Q407857""","""IKGXIBQEEMLURG-NVPNHPEKSA-N""","""InChI=1S/C27H30O16/c1-8-17(32)20(35)22(37)26(40-8)39-7-15-18(33)21(36)23(38)27(42-15)43-25-19(34)16-13(31)5-10(28)6-14(16)